# 📖 Chapter 05 — Conversation Memory & Source Citations
## 🎯 Objectives
Enhance RAG system with conversation memory and source citations.
**What we'll accomplish:**
- Add conversation memory for multi-turn dialogues
- Implement source citations for transparency
- Test and compare improvements


## 💬 Part 1 — Conversation Memory
### Step 01 — Setup & Load RAG Chain
Import libraries and load the basic RAG chain from Chapter 4.

In [10]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough, RunnableWithMessageHistory
from langchain_core.output_parsers import StrOutputParser

from src.rag.embeddings import create_embedding_model
from src.rag.vector_store import create_vector_store
from src.rag.llm import create_llm
from src.rag.prompts import get_default_rag_prompt
from src.rag.rag_chain import create_rag_chain
from src.utils.emoji_log import done, info, success, task

info("All libraries imported successfully!")

💬 All libraries imported successfully!


### Step 02 — Load Basic RAG Components

Load the vector store, LLM, and create a basic RAG chain (without memory first).

In [ ]:
task("Loading RAG components...")

# 1. Load embedding model
info("Loading embedding model...")
embeddings = create_embedding_model()
done("Embedding model loaded")

# 2. Load vector store
info("Loading vector store...")
vector_store = create_vector_store(
    collection_name="travel_attractions", embeddings=embeddings
)
done("Vector store loaded")

# 3. Create LLM
info("Creating LLM...")
llm = create_llm()
done("LLM created")

# 4. Get prompt template
prompt = get_default_rag_prompt()

# 5. Create basic RAG chain (without memory)
info("Creating basic RAG chain...")
basic_rag_chain = create_rag_chain(
    llm=llm, vector_store=vector_store, prompt=prompt, k=5
)
done("Basic RAG chain created")
success("All components loaded!")

🚀 Loading RAG components...
💬 Loading embedding model...
🏁 Embedding model loaded
💬 Loading vector store...
🏁 Vector store loaded
💬 Creating LLM...
🏁 LLM created
💬 Creating basic RAG chain...
🏁 Basic RAG chain created
✅ ✅ All components loaded!


### Step 03 — Test Basic RAG (Without Memory)

Test the basic RAG chain with a multi-turn conversation to see what happens without memory.

**Expected behavior:** The chain will NOT remember previous questions.

In [4]:
task("Testing basic RAG without conversation memory...")

# Question 1: Ask about Space Needle
print("=" * 70)
print("Question 1: What is the Space Needle?")
print("=" * 70)
answer1 = basic_rag_chain.invoke("What is the Space Needle?")
print(f"Answer: {answer1}")

# Question 2: Follow-up question using "it"
print("=" * 70)
print("Question 2: How tall is it?")
print("=" * 70)
print("(Without memory, the chain doesn't know what 'it' refers to)")
answer2 = basic_rag_chain.invoke("How tall is it?")
print(f"Answer: {answer2}")

# Question 3: Another follow-up
print("=" * 70)
print("Question 3: What else is nearby?")
print("=" * 70)
print("(Without memory, the chain doesn't know we were talking about Space Needle)")
answer3 = basic_rag_chain.invoke("What else is nearby?")
print(f"Answer: {answer3}")
info(
    "Notice: Without memory, the chain cannot understand context from previous questions!"
)

🚀 Testing basic RAG without conversation memory...
Question 1: What is the Space Needle?
Answer: The Space Needle is an observation tower located at Space Needle, 400 Broad Street, Seattle, WA 98109, United States of America. It is an icon of Seattle, a designated Seattle landmark, and was built for the 1962 World's Fair in the Seattle Center. It is 605 ft (184 m) high and features an observation deck 520 ft (160 m) above ground, offering panoramic views.
Question 2: How tall is it?
(Without memory, the chain doesn't know what 'it' refers to)
Answer: The Seattle Great Wheel is 175 feet (53.3 m) tall and is located at Pier 57 on Elliott Bay in Seattle, Washington.
The sculpture "Untitled" measures approximately 19 feet (5.8 m) x 14 feet (4.3 m) x 10 feet (3.0 m) and is installed at Louisa Boren Park in Seattle, Washington.
The Space Needle is 605 ft (184 m) high and is located at 400 Broad Street, Seattle, Washington.
I don't have height information for Duwamish or The Eagle.
Question 3

ServerError: 503 UNAVAILABLE. {'error': {'code': 503, 'message': 'The model is overloaded. Please try again later.', 'status': 'UNAVAILABLE'}}

### Step 04 — Add Conversation Memory

Now let's add conversation memory to enable multi-turn dialogues.

**Key Components:**
- `ChatMessageHistory`: Store chat messages
- `RunnableWithMessageHistory`: Integrate memory with chain
- Session management for conversation tracking

In [8]:
# 1. Create a store for chat histories
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

In [ ]:
# 2. Create a new prompt template that includes chat history
conversational_prompt = ChatPromptTemplate.from_messages(
    [
        # This is a tuple, tell AI the system message
        (
            "system",
            """You are a helpful travel assistant specializing in tourist attractions.
Use the following context to answer the question. The context contains information about various tourist attractions.
Context:
{context}
Instructions:
- Answer based ONLY on the information provided in the context above
- If the context doesn't contain relevant information, say "I don't have information about that in my database"
- Be concise and helpful
- Use the conversation history to understand context and pronouns""",
        ),
        # This is a placeholder here to insert the chat history
        MessagesPlaceholder(variable_name="chat_history"),
        # Another tuple, user message
        ("human", "{question}"),
    ]
)

In [7]:
# 3. Create retriever
retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [9]:
# 4. Build conversational RAG chain
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


conversational_rag_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
        "chat_history": lambda x: x.get("chat_history", []),
    }
    | conversational_prompt
    | llm
    | StrOutputParser()
)

In [11]:
# 5. Wrap with message history
conversational_chain_with_history = RunnableWithMessageHistory(
    conversational_rag_chain,
    get_session_history,
    input_messages_key="question",
    history_messages_key="chat_history",
)

## 📚 Part 2 — Source Citations
### Step 05 — Return Source Documents
Modify RAG chain to return source documents.

### Step 06 — Format Citations
Create citation formatter for clear source display.


### Step 07 — Test Complete System
Test the enhanced RAG with memory and citations together.

## 📋 Step 08 — Chapter Summary
Review achievements and prepare for encapsulation.